<a href="https://colab.research.google.com/github/Saim-Hassan786/Agentic-AI-With-OpenAI-Agents-SDK/blob/main/11-RunnerContext/RunnerContext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RunContext
RunContext is the **context** that we pass to the **Runner** in the **context** parameter this context is not passed to the LLM and is used by our implemented code tool fucntions, hooks callbacks etc

In [ ]:
# Installing the SDK
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00


In [ ]:
# For running event loop
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Pre requisites SetUp
from google.colab import userdata
GROQ_API_KEY= userdata.get('GROQ_API_KEY')

from agents import set_default_openai_api,set_default_openai_client,set_tracing_disabled
from openai import AsyncOpenAI

external_client = AsyncOpenAI(
    base_url = "https://groq.helicone.ai/openai/v1",
    api_key = GROQ_API_KEY
)
set_default_openai_client(external_client)
set_default_openai_api("chat_completions")
set_tracing_disabled(True)

In [ ]:
from agents import Agent,RunConfig,function_tool,RunContextWrapper,Runner
from pydantic import BaseModel

class Context(BaseModel):
  name: str
  age: int

@function_tool
def greet_user_with_context(ctx:RunContextWrapper[Context]):
  return f"Hello {ctx.context.name} you are {ctx.context.age} years old"

agent_with_context = Agent(
    name = "Context Agent",
    instructions="Greeting Agent",
    model = "llama3-70b-8192",
    tools = [greet_user_with_context]
)

result_with_context = await Runner.run(
    agent_with_context,
    "Can you greet me pls",
    context = Context(name="Saim Hassan",age=25)
)

In [ ]:
print(result_with_context.final_output)

Hello Saim Hassan, you are 25 years old.


# RunContextWrapper
It wraps our context and returns 2 objects:

1. **context**: this takes our oroginal context that we want to wrapped.
2. **usage** : the total usage of the Agent Run so far.

In [ ]:
# context
print(result_with_context.context_wrapper.context)

name='Saim Hassan' age=25


In [ ]:
# usage
print(result_with_context.context_wrapper.usage)

Usage(requests=2, input_tokens=1823, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=55, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=1878)


# Usage
This usage attribute contains following features :

1. **requests** : the number of requests sent to the LLM
2. **input_tokens** : the total tokens sent to the LLM
3. **input_token_details** : the details of the input_tokens sent to LLM with **cached_tokens**
4. **output_tokens** : the total tokens received from the LLM
5. **output_token_details** : the details of the output token recieved from LLM with **reasoning_tokens**
6. **add( )**: to add Usage of other agent run

In [ ]:
from agents import Agent,RunConfig,function_tool,RunContextWrapper,Runner
from pydantic import BaseModel

class Location(BaseModel):
  city : str
  country : str

@function_tool
def user_location(ctx:RunContextWrapper[Location]):
  return f"The user is from {ctx.context.city}  {ctx.context.country}"

agent_with_context_2 = Agent(
    name = "Context Agent 2",
    instructions="Assistant Agent",
    model = "llama3-70b-8192",
    tools = [user_location]
)

result_with_context_2 = await Runner.run(
    agent_with_context_2,
    "Can you tell me my location , myself Saim",
    context = Location(city="Lahore",country="Pakistan")
)

In [ ]:
print(result_with_context_2.final_output)
print("=========="*20)
print(result_with_context_2.context_wrapper.usage)
print(result_with_context_2.context_wrapper.usage.input_tokens)
print(result_with_context_2.context_wrapper.usage.input_tokens_details)
print(result_with_context_2.context_wrapper.usage.output_tokens)
print(result_with_context_2.context_wrapper.usage.output_tokens_details)

So, Saim, it seems that you are from Lahore, Pakistan.
Usage(requests=2, input_tokens=1817, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=55, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=1872)
1817
InputTokensDetails(cached_tokens=0)
55
OutputTokensDetails(reasoning_tokens=0)


In [ ]:
# adding the both usage
result_with_context_2.context_wrapper.usage.add(result_with_context.context_wrapper.usage)
print(result_with_context_2.context_wrapper.usage)

Usage(requests=10, input_tokens=9109, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=275, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=9384)
